Сначала нужно определить какие столбцы мы хотим загрузить из нашей таблицы. 

Так, для моего примера мне нужны столбцы Volume(unit)(2), SurfaceArea (um2)(5) + количество строк.

Шаг1. Извлечение данных из xlsx в df, с которыми я смогу работать в python

glob - показывает пути к файлам с указанным расширением и сохраняет в переменную filepaths

In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 

import pandas as pd
import numpy as np
import glob

Функция csv_to_xlsx считывает все файлы csv в папке, затем создает xlsx для каждого файла

In [ ]:
def csv_to_xlsx():
    csv_search_string = '**/*.csv'
    csv_filepaths = glob.glob(csv_search_string, recursive=True)
    df = pd.DataFrame()
    for csv in csv_filepaths:
        out = csv.split('.')[0]+'.xlsx'
        df = pd.read_csv(csv, sep=';', encoding='latin1',engine='python')
        df = df.to_excel(out, float_format="%.4f")
    return (df)
#csv_to_xlsx()

создаем пустой датафрейм, куда будут загружаться считанные столбики таблиц

в цикле объединяем датафреймы из считанных файлов 

Нужно изменить названия столбцов,чтобы было очевидно их происхождение из другого файла. 

Для этого была написана функция columns_rename, где в цикле добавляется порядковый номер файла, из которого происходило извлечение


Далее заменяем измененным названиями столбцов действующие названия в датафрейме.

In [ ]:
def file_extraction(search_string, N, stack_num): 
    filepaths = glob.glob(search_string, recursive=True) #можно изменять номера в N1*stack1,чтобы извлечь другие файлы
    df = pd.DataFrame()
    for find_files in filepaths:
        df = df.join (pd.read_excel(find_files, usecols=[2,5]), how='outer',lsuffix=' ')
    
    columns_names = [j for j in df.columns.tolist()]

    def columns_rename(columns_names):
        j=1
        for i in range (0, len(columns_names), 2):
            columns_names[i] = 'N' + str(N) + '_stack' + str(stack_num) + '_cell' +  str(j)+ '_' + columns_names[i] #можно изменять номера в N1*stack1,чтобы отметить 
            columns_names[i+1] = 'N' + str(N) + '_stack' + str(stack_num) + '_cell' +  str(j)+ '_'  + columns_names[i+1] # принадлежность к другой партии данных
            j+=1
        return(columns_names)
    df.set_axis(columns_rename(columns_names), axis=1, inplace=True)
    return(df)

Извлекаем в отдельные датафреймы все файлы по стекам

(?) подумать как можно оптимизировать, т.к. по всем животным будет под 50 стеков, прописывать каждый скучно

In [ ]:
N1_stack1 = file_extraction('**/N1*stack1*.xlsx', 1, 1)
N1_stack2 = file_extraction('**/N1*stack2*.xlsx', 1, 2)

N2_stack1 = file_extraction('**/N2*stack1*.xlsx', 2, 1)
N2_stack2 = file_extraction('**/N2*stack2*.xlsx', 2, 2)
N2_stack3 = file_extraction('**/N2*stack3*.xlsx', 2, 3)
N2_stack4 = file_extraction('**/N2*stack4*.xlsx', 2, 4)


Посмотрим на значения основных критериев описательной статистики

In [ ]:
N1_stack1.describe()

Извлекаем отдельно столбцы, анализ которых будем выполнять.

(?)из-за присутствия NaN не считает перцентили (если заменить 0, могут сместиться значения исследуемых критериев). Подумать как это можно разрешить, м.б. при рассчетах можно указать использование значений >0.

(?)попробовать автоматизировать создание столбиков нужного вида 

In [9]:
N11_cell_volume = N1_stack1.iloc[:, 0:4:2].copy()
N11_cell_surf = N1_stack1.iloc[:, 1:4:2].copy()
N12_cell_volume = N1_stack2.iloc[:, 0:4:2].copy()

#N11_cell_volume2 = N11_cell_volume.fillna('')
#q25, q75 = np.percentile(N11_cell_volume2.iloc[:, 0],[.25,.75])

Попробуем сделать общую таблицу по всем животным с 2 столбиками.

Для этого нужно создать какой-то цикл или функцию, извлекающие нужные столбцы и создающий на выходе датафреймы с нужными столбцами

all_cell_vol = N11_cell_volume.append(N1_stack2.iloc[:, 0:4:2]) - использовать присоединения столбиков


не понимаю как посторить гистограмму распределения для всех столбиков((

In [83]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.distplot(all_cell_vol(N1_stack1_cell1_Volume (unit) ) )
#all_cell_vol = pd.concat([N11_cell_volume, N12_cell_volume], ignore_index=True, sort=False)
#all_cell_vol = N11_cell_volume + N12_cell_volume
#all_cell_vol.fillna(0)
#plt.hist(all_cell_vol, bins = 4)
#all_cell_vol[all_cell_vol > 0.01].hist(bins=10)
all_cell_vol.columns


NameError: name 'N1_stack1_cell1_Volume' is not defined